In [2]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

/home/tarin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
path = 'poem.txt'
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

corpus length: 74136
total chars: 444


In [4]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 24699


In [5]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [6]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [7]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [8]:
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [9]:
model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Epoch 1/60
24699/24699 [==============================] - 36s 1ms/step - loss: 3.9475

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "する。みゝなれ侍にけりやときこえ給ふ。
△△みや人にゆきてかたらん山桜かぜよりさ"
する。みゝなれ侍にけりやときこえ給ふ。
△△みや人にゆきてかたらん山桜かぜよりさきこのはかなしに／かはからむ。とのこころも。と
△△いかならぬ／かなるたのかきのかなしにものこのこともしきこころもからしきしらん。と
△△いかさらむ／かにしらに／はすかれに／かきかたるはなきこころもかきしかなきしのこころも。と
△△かきこともあまかなきこころもともしきこのこともかなしき。とこころもかなし。と
△△いとのころももはかたるかしにものかきみにあまし。ともひとこころも。とのかきたま
△△みたらしものあれかきしきのあるかたにかきのかきなき。とのこともははなき。とのみり
△△いかならしものころもともしらぬはかなしとのかにかたる。とのこころも。とのこころも。と
△△いとからてものかくかたるかきみかたにはかきこころもからしきみのかなしきこころも。とに。と
△△かきたましもともかたらぬらん。ともともかきこころも。とのこころも。ともあまへり。いと
△△あからしはかににかならぬかきこころもかきの
----- diversity: 0.5
----- Generating with seed: "する。みゝなれ侍にけりやときこえ給ふ。
△△みや人にゆきてかたらん山桜かぜよりさ"
する。みゝなれ侍にけりやときこえ給ふ。
△△みや人にゆきてかたらん山桜かぜよりさしの／かけるもたるのまやとにたちふらぬと。いま
△△かきりなるすとのかなわかなのことこかかたるみたはなそもに。ももはなまし。とうらしん。おもなき。。と
△△かくりてもとのかくたれたましきのあるならぬのかにな
△△かきことものかきこころははれしらん。こころもかける。おんかきのみなりにあるはなる
△△からにもいそのみをかきこおもふきはししものおはひとも。ひとたまめける。とのあらる。と
△△たきへとありにこはからぬほのことの

△△かけひばやときりとめをるをふふてもいはそ身もおもひんどのこゝろにやうで年波うら
△△鳥にしらはねにかふるさめかなりつゝかみねとものゝ
△△なからんやねびかぎはがらむくしめにかたりにしつてのぢのまほとあさくとてふるこゝろへや
△△をげれぬる聞えどにみれよめみりけまほ。あれば花のそそおやとやとつめてば
△△松ぼらすすしたちきやもい山にみらずさへるえぞぬるゝはすり。まぞすや。とのきえずて
△△ふづればて々＋＋つるまさふの雨きをはれ給へつらしきぢゐがよりなきつゝかられどは。
△△草のゝづはたづのにわかれぬとやかせだとをほつゝせたるゝいづれなだしとめ
△△いきにとみねそればかりもねものこころもゑな
----- diversity: 1.2
----- Generating with seed: "きとられぬるおひなれはかくてたえぬるなかとかこたむ。＊えのかれたまはしとあり（＊"
きとられぬるおひなれはかくてたえぬるなかとかこたむ。＊えのかれたまはしとあり（＊はのぼ音にてなれ（＊ゑたろとしよるんとちと
△△わかさゝてしばそりともとと人に△よげやたにのふごえぬ世にひいずつゝた御あくしきころし
△△露の浦ぬらひてしの思ひやかなるゝをはぎかいしをなしくゐこぎにあるらんの我し。たれに
△△おくへずこ＊じをくれはうめしごをきよりももなるあもしくの人つはつき。をれねしぎの御うれ給山もろれに
△△は衣なみばかた＋るふ習だはじ松どこせしきから御ひよししこ又かり。御てんとひ給ふ
△△葉の木ら子はたなのはゐにか深ちをかつらひしたる道ゞへ共ぢをいろなに
△△＊おぼにうつとみしくみよふかなみおぼ春に／こゝろめだきじ。せん
△△てながめたふにちにたらりの世ちりのんぶそめのさのめおしをうらあけやそれそ聞える
△△さむとしき従にいけすつなかはひともおきたけゐぬむましきた。ここそへ
△△にくたにもわれかもくちかかきひなすことはそみつるまあさき。ふきみをしほらぬあ。いあらひとご
Epoch 5/60
24699/24699 [==============================] - 35s 1ms/step - loss: 2.8293

----- Generating text after Epoch: 4
----- diversity: 0.2
----- Generating with 

△△なからむるはちのちにうたひすてあさしはなかきみつもなくらひなれて。いのかへり。とて
△△かけてもとこそかはれなるかなきはなかれのみしかなきこえさらは。なにいかにまさるあまひとも。と
△△よの
----- diversity: 0.5
----- Generating with seed: "こそをしけれ。と＊いとをかしけれは（＊いとほしけれは）
△△ふかからすうへはみゆ"
こそをしけれ。と＊いとをかしけれは（＊いとほしけれは）
△△ふかからすうへはみゆきかしともみからぬにもなれてもたちかりすちのしき。と。おんかはと
△△くものえはいひしこそみれいとにかくらしくもゐのそらになきみし。まん。とのたまふもありやま。はつまひかけて。
△△よのつともいかかともひとあらぬよにあかけてもたかかれにいとゞたれて。はなのかりの）。とあるさま
△△きみにまてふるひなりけるわかれとはたれかはるらむここちはむ。ことひとかたきこころに。つきても
△△わかれてもそへたつねてもはかなくもなほれぬるかなきみなときむ。かきひたまへりたまへり
△△おもひとのあふかはこるもしらつゆむときりともおもふかたちにたま。あまのはなにかけてもたまふ。かんな
△△かすならしことはかたににふかけてはなかれとはなきかな。とをたまへはせても。とのこひたまは。とあるをこひかは。いと
△△わかれしもきかかりにしるけふりをしらむなけきしのかは。とひとりこそいたて。とはかひかけて。いと
△△なけ
△
----- diversity: 1.0
----- Generating with seed: "こそをしけれ。と＊いとをかしけれは（＊いとほしけれは）
△△ふかからすうへはみゆ"
こそをしけれ。と＊いとをかしけれは（＊いとほしけれは）
△△ふかからすうへはみゆせしことのつかりにはののおひとのすきをそらしと。おんてのとりて。とのいひは。いと
△△くみたせはきみのおたひままひせしまたなすむかにころしもてそむき。おさしあはきつゆふきこえたまへむここり
△△りかけしきわかれしほとにあまつかむかなしめむすははかなかめしと。すにかきす
△△しくさへにそへるそらにはなみおほたこそかなしきはおやと。さきこそめたはおぼしをゆふし
△△かたくれにそきなかきあるけふしきをわからしたる名ひさかなへす。かにしきこえたまふ。かんり
△△わか山しもおほか

△△つらわれ給へだるそまどちはふるしよにもあへさりみて。なんのつのぼのとにの御わすろひて。
△△かきなこはこころなとてむかずならむいのすこにしくいそわひて。はかなしきそてか＊なをたひてやつきか
△△みきひととつゆめくしらを御て月けふもかれはねてすくつゆのなほなるめとおほたけむ。こころへり
△△なくらねなふかは成す。よきうちはたたすくさへてほとはいよ。けにやかわたひなくさへは。れて
△△くそ立思をいとふためさきにかみたましをしまつるなかなくはむつきをくひともなき。ひいはじ
△△ほこひさるみくこひすえをやと＊はなぢよはなくほとなともゆくつゆらふたかならむ
Epoch 12/60
24699/24699 [==============================] - 37s 1ms/step - loss: 1.8930

----- Generating text after Epoch: 11
----- diversity: 0.2
----- Generating with seed: "るらん。をそくはびんなからん
△△むかしこそまづわすられねすみよしの神のしるしを"
るらん。をそくはびんなからん
△△むかしこそまづわすられねすみよしの神のしるしをかけてこそかへりて。とのかはからにとある
△△なけきわするることのやまをたつかしはかきはるらしもこそいかにかなしき。いとつかはなにいと
△△したふかきあさきしのひともなくらしをしるらしはしのみしまめやは。ときこゆ。いろふきかきたる
△△かきつゆてきみかへてなきはつねをいかにつすならひとりごとしるるのとかなきこと。なに
△△おほかたにいつともなきひとのあはれをみちのまとはまのかきひとのかき。なさきおもほえさる
△△うきふしもしほたれるよをたのみつれともいかにかくりたりこそ。れゐのかはをならぬなれれ
△△かくりのかけははなきはなれきかけさまになほほたるるのかみのやまをおほしける。かに
△△おほかたにいつともなきよをつらしきこのはれはかなききのつきかき。うつすきほたる。こんのうさにおちすてしたまふ。おんさかなるしとも
△△かきつゆてきみかともみをなれむとこよをしこよをかくりたるそふ。なほたりにし
----- diversity: 0.5
----- Generating with seed: "るらん。をそくはびんなか

△△いかはらむなみのうちにろしもともなくさねはかけはのれともみや。とのたまへり。たまふ。
△△＊みたはみそこれをためしとなきものは
----- diversity: 1.0
----- Generating with seed: "あらしふくをのへのさくらちらぬまをこころとめけるほとのはかなさ。いととうしろめた"
あらしふくをのへのさくらちらぬまをこころとめけるほとのはかなさ。いととうしろめたまへ
△△ゆくさすふはなのしよりにかけはなれくさひなき＊おまへをしてなと。すさまん。わかふためたどとはやる
△△こくひにてつゆかつむへたむすひとかたきはのはをせのひとかたにとふふき。みつかたにいとれる。とかきて此け
△△かくかりのふきのたちしはいせやるきみかなくそのそらけれるみつとこな。はまひともせぬ。いかにまかり
△△みるさへのやみしきぬらにしほとたむかひなかなさめしなりぬは。こかよたにつとならいとけるを。に
△△いかさらむかつみともむらひにしよにつらなればわかぬらひきかさねにやは。とのたまへは
△△かゑのこのあはれをたちむのころもならははのうしはまもはちのそむ。なにたれとこさまかなみたそぬれなき。いとそういひたまへり。こころにうすみるけしと
△△きみとはむくもゐのねけもたれかたもそむすゑもわれやしくはむ。こかなとしたまへるを。いとゆれは
△△かくさくむこのうへよにのことをそこひしつるく
----- diversity: 1.2
----- Generating with seed: "あらしふくをのへのさくらちらぬまをこころとめけるほとのはかなさ。いととうしろめた"
あらしふくをのへのさくらちらぬまをこころとめけるほとのはかなさ。いととうしろめたる
△△あさひさしけふはそこころやしもにつこそやとそあはれそへな（＊こたるつまがむ。おなろ
△△かこなしむ身をかきて覚のこかなけきそのれはか＊のよのつけてもへらむさよともすゑとも
△△とくさなくあひたのころもきかけれとやみかくさつるした木はな（そつれに。とのたまへる袖は
△△れぬうしふ風ふせにしや△△神さなじさ木のはなれしむねのきかめそおりかわかくふからこはぬり。うち給ありも。と
△△あさきらむしもひすこすもほそよたにうつわらやかくすへらぬものかなくえり
△△わたれすのあらのなかりはへぬるはおりけにひとをみえぬあけそなさゐてわたるご

...

△△うきはらていものうらひぬあきにしこたそふねのまくものはせぬ。といとわかなくらんとをしも。かきたれに。この
△△ほのかににかよふはなかれたちならむなかのとともなしきを。はなにていとをちすてしれ
△△かけはてはかそなききえしまたをくもひはあらそふるやかなる。とうちすてたるをり。いと
△△みしかはあはれのうちにしわかかひかれもいかにつけてもはかぬちきりや。かくたりけり。
△△そてのこのあくみかやみをたちかさねきしきのよをはむ。このやみかきくたえやとかりを。ちきひかたお
△△かけてよのつきかは
----- diversity: 1.0
----- Generating with seed: "ひひきにねをそへてわかよつきぬときみにつたへよ。くわんしゆもてきたるに。かき
△"
ひひきにねをそへてわかよつきぬときみにつたへよ。くわんしゆもてきたるに。かき
△△とまでみちをきしつねもかきくらむなかのころもをかすならすな（＊たよまこひすをすて
△△かけはさまふねやみのかけはえすすみのおもふをく人にやまらむるかなきくことはこよそ
△△しめのはをみちにつるはもこのこそかへるをぐしうしほのかみそてわたつれ。なほほどたて。
△△そてちそかべしゆきとしめへる／心をあらば／しらぬ
△△山づはへり／あさそぎくれど／れどりぬらん。わかなと。たたはことにたきおほかへることなる。ひとけれたるやはづらをたれへ
△△ふるとそのはるみはねをとはかそてもかけはなれきむすどのしられば思ひなること
△△いかなともみを人づやこふあまの月いまでやひとおぼさるゝはる人のおもほどひやひて
△△こまなみはやまあらふにゆるらんて身にかゝるを風になりさまなくえは。とえがたにてに。との御かどたは。
△△ん。おなじくなかなるほどにをく霜はまの／
△△るそたこ／たひろきにけん／むらんと
△△ほど花
----- diversity: 1.2
----- Generating with seed: "ひひきにねをそへてわかよつきぬときみにつたへよ。くわんしゆもてきたるに。かき
△"
ひひきにねをそへてわかよつきぬときみにつたへよ。くわんしゆもてきたるに。かき
△△＊はか夢さきあらしよころもたのかとおりはこめわれたま鳥。なれはみたるとば。いさとわれひみそん
△△うきふらむなしにもこたるるゝちきかたのみきここゆる共もあるやうつらし風をや

△△なほさりにたのめおくめるひとことをかへしきのかけはたのかきあさめむとおときけす。しる
△△なしひとをいととつけてもいかのまつにのうらもほかりはきみかたるあさそなき。と。うちわたりしかはとぬめき
△△はつかりにはなのあるれもしらすきみえぬつねならぬなかなきこと。さかなるはきたのこころなか。
△△とことはまくかははせさへにてくもゐになかなほくふかみをはとむする。なかなほにやまひにもあますすむつる
△△うらにとりあけたしきにしらねはなくさそらむなかめしけやうちのまつ。とかきていつをつけて。
△△そてふかたねとはみにしるけはたちならはなのあさにこゝろひとける
△△つらなさのこゝろもしらをしつねてもわれかぬやけるかなんなかなからむことはかくそて
△△はかりみそはなかくしげにねかせさきてなりねとしるあめのかなしけれ。と＊みか
△△ほしめみるこのふくさきのこころにしましあけやまのかもはのあらたきにしらすき。えつかしきことに
△△いかは
----- diversity: 0.5
----- Generating with seed: "きうらみたにせし。あはれにうちなきて。こと
△△なほさりにたのめおくめるひとこと"
きうらみたにせし。あはれにうちなきて。こと
△△なほさりにたのめおくめるひとことをかへしきのかけはたのかきりとめしきも。といとわれたまへらむ。
△△つれなとていはのかみにゆるしけましはかたのまつなとこころなりけれ。とて
△△からのなみはたれかなくちのかせのはかりのはなくてかなるあきくそ。とやなつつな。いとりていとわか
△△はかはなきはなのたよりにみころつゆきちきりかきそらてなれやまさす。これはむするかなひとつけも（＊はちなかつみのかみのまさにさまて。いとそつしかはありきかなしき
△△いくらしてふえぬことはれおぼしかたちとはみぬやとなりなれるよと。なに。といひわまひけり。かみのかみにありけり。と。みち
△△かきつめてみるものをりはむすほほれあけふくこちきかけさり。とうちみたりたるおほしりなと。
△△かくひとのたつねにやとをたにかけはまつるまたつゆのひなかさのこころに。いまふかにこころに
△△うつしまのねをかたもせむふるあさをもこまやまのはなくこゝろひへり。なんのねをたた
----- diversity: 1.0
----- Generating with seed:

△△はせのしのおもひきてかたにしらつむまつかのをわられたはそらぬほのこころはそりて。つのみつれたる空にうちきみえたたまへは。いかなくしも
△△おひかきすたけふひきをむするるつきか身もひともかとはかや浦。と。なほたりかきたち
△
----- diversity: 1.2
----- Generating with seed: "しほどけしとや人のいとはん。とあるを御らんじつけて。さは
△△かたみにぞかふべか"
しほどけしとや人のいとはん。とあるを御らんじつけて。さは
△△かたみにぞかふべかりけずたきめしもうきわすれねはおもほえさまのゆゑ
△△すりもえはかはれぬてみがたちおひすきはしやくのませたのまつつかなさへ。さてわたい
△△みや人のねならうたらむれわひかしとたれるみ君にながめんらん思ひやうちげにゐのみつかなきこえんこそそらづくもしきこみはな心ぬなむ
△△たけふかもいはゞうたしきえつらしきはしまよりとあらはしころにそへぬける。きねとお
△△つねならずはこちわたらしにいかにますせていかにまけるのを。ときかはとおほすらつかなきたま
△△いのとならむこともふかくもおもひしやまつくもきことたひかつきおもひしのふきやまと
△△わかれしうへいてふかためおととやまははのくもゐにゆふしか。けさりにまとこと。とめやうつらしたまふ。おんたうのおんこりのみまけと
△△かけきおとちかはつるさとあまよふきによるへきてもなたえめきよに。とあきこと。とかやうき
△△からわさてみももとおりもたむおひなきに
Epoch 52/60
24699/24699 [==============================] - 35s 1ms/step - loss: 0.7187

----- Generating text after Epoch: 51
----- diversity: 0.2
----- Generating with seed: "△ありふれはうれしきせにもあひけるをみをうちかはになけてましかは。うちゑみたるを"
△ありふれはうれしきせにもあひけるをみをうちかはになけてましかは。うちゑみたるを。あとりなしくこゝろ
△△すすんてはわかかけふよのうきよをしててこころをくもるはなをいひしれるけの。まかれ
△△かきつけのちききをみしもとめてしけりみやまのはにうきをにはすむ。いのちのありを

△△をしかなく／秋の山里／いかならん／こ萩が露の／かゝる夕暮／たゞ今のみいとゞへさまもいか身こころなり
△△をたづらねき／身こそけに／たれやどり／／をけん。おほやどにもうきゐのうちにまがひなきことをおもひ
△△ふかげりに／みえましはせ／たれやまするかにとある物など。とめてもと＊なきたつけす
△△あまのしもひとこゝろしむよのしらにこころをそくむつらすゑかすて。ひとりなりきこえたまふ。いと
△△ひかつらてしたふるつねのうちにちかへてことろこそけきなつねをいはとらむ。とむかたうつきしとけし
△△おほかたにくれをみやめむことしかたにききかためるにはころも。とありや。いでともあれたる
△△あささらす
△△かきりけるきししもなほたれかいふにより
△△やまはたの／おほかこころにそこよろしてことのかりかりやせん。なりて
△△かくもとをつゆきやとりをいつとしれける／そくれどをこける／したのかな。とよをたりしこりせん。おんさうのかにおほにたる＊みた
△△ほのころにそこもふるその
----- diversity: 1.0
----- Generating with seed: "やけごと
△△をしかなく／秋の山里／いかならん／こ萩が露の／かゝる夕暮／たゞ今の"
やけごと
△△をしかなく／秋の山里／いかならん／こ萩が露の／かゝる夕暮／たゞ今のみちに／むえもとたせぬなきかな心に／こゝろづは）て。いとひとのこひたつこれは。。
△△みだにしてわすれしらははおものまどふきからみしのあるやまや。あるのやまたまたとあり。いと
△△いにそへてをきみつとおもふはぬれなこはすれなれやおとろきやかにそており。こんまとのつけて
△△あを舟れのいもまつつけをちけつとしよのつゆそちるにもあまとも。とあるやうへすくもは。みたるたまひなき。ひねて
△△かはのはをうすにこころとともきりあへくてにやまめにわかちきせのていといかしとつめ
△△とみしたるいろのうつにいひわすりもととすきえしこひわすらぬれ。とそらたつきそてのひひたりやとひこりしか
△△はるのめてきしからむつゆのしつらむかさのうらもやかなさふ。と＊いひかたつけておへたまへり。ことはかよからなるあめり
△△わかやとのいもとはしなをしるらすてはなしていえのかみには。ととかきくらしやと。いろにかすきし
△△
----- diversity: 1.2
----- Gene

にまけむ。とあかきおもはすりし
△△かけたえむこのかへしけきかなしのうしはてに。む＊ことうきひけにたつきしとやみつれ
△△ここのとそちきりしものかむえならぬなにこころにかけると
Epoch 59/60
24699/24699 [==============================] - 40s 2ms/step - loss: 0.6439

----- Generating text after Epoch: 58
----- diversity: 0.2
----- Generating with seed: "わするる。とはちらひてきこゆ。
△△おほかたはおもひすててしよなれともあふひはな"
わするる。とはちらひてきこゆ。
△△おほかたはおもひすててしよなれともあふひはなれしたつやうつらむ。とうたおもはすら
△△なきたれはみほとをしらぬむすむかなほくにもほとそゆくへきみし。はありうし。しよならへ
△△しほかれしあれやしよをつむなのこともきりてきこえしかとならぬあつましき。と。うちなきたけに
△△あさかさをはちにつつねはたよふみよりにすすほとにおとしのややは。いつれにつゆふしきこえたまへは。おほかひみちにてちなとめて。もののみはや＊いのわくとしらぬるを。かのまことも
△△あはれてをそらかなもえむおほそらひおまのやまさるいとなれやみるにも。をかしとやり
△△しほかれにあるまをしらぬしにこころなとなれきこころはそと。なけなと。みたまふ。もの
△△かきつらてむかしのこらぬことふかきみをわすれしははれをとふみむ。とのたまへと。なほなりに
△△のちかさにはなにこころをしものなかれきたつねはたれかとけれと。いといつとおしとややささま
△△ことのうらしそかれとおとへしるみ
----- diversity: 0.5
----- Generating with seed: "わするる。とはちらひてきこゆ。
△△おほかたはおもひすててしよなれともあふひはな"
わするる。とはちらひてきこゆ。
△△おほかたはおもひすててしよなれともあふひはなれしこころととより。とあることなる。
△△かくひおもいはのうらもはこしけきあかたえにあかみにここるわかなく。かくそおほしるるはき。
△△つらひけりゆふふちちりはあられともおもひなりねとみえたけはなれやいとしるのと。あほしり
△△わかやとのは